In [ ]:
import { z } from 'zod';

const stringSchema = z.string();
const numberSchema = z.number();
const booleanSchema = z.boolean();
const stringArraySchema = z.array(z.string());
stringArraySchema.parse(['apple', 'banana', 'cherry']);

const personSchema = z.object({
  name: stringSchema,
  age: numberSchema,
  isStudent: booleanSchema.optional(),
  home: stringSchema.default('no home'),
});
// stringSchema.parse('Hello, zod!');
stringSchema.parse('12341');

const mixedTypeSchema = z.union([z.string(), z.number()]);
mixedTypeSchema.parse('hello');
mixedTypeSchema.parse(42);

In [1]:
import { z } from 'zod';
import { zodToJsonSchema } from 'zod-to-json-schema';
import { Ollama } from '@langchain/community/llms/ollama';


const getCurrentWeatherSchema = z.object({
  location: z.string().describe('The city and state, e.g. San Francisco, CA'),
  unit: z.enum(['celsius', 'fahrenheit']).describe('The unit of temperature'),
}); 

const paramSchema = zodToJsonSchema(getCurrentWeatherSchema);
console.log('ps--', paramSchema);

const model = new Ollama({ 
  baseUrl: 'http://localhost:11434',
  model: 'llama3.1:8b',
  temperature: 0,
});

const modelWithTools = model.bind({
  tools: [
    {
      type: 'function',
      function: {
          name: "getCurrentWeather",
          description: "Get the current weather in a given location",
          parameters: zodToJsonSchema(getCurrentWeatherSchema),
      },
    },
  ],
});

await modelWithTools.invoke('北京的天气怎么样');


ps-- {
  type: "object",
  properties: {
    location: {
      type: "string",
      description: "The city and state, e.g. San Francisco, CA"
    },
    unit: {
      type: "string",
      enum: [ "celsius", "fahrenheit" ],
      description: "The unit of temperature"
    }
  },
  required: [ "location", "unit" ],
  additionalProperties: false,
  "$schema": "http://json-schema.org/draft-07/schema#"
}


"北京的天气多变，夏季炎热干燥，冬季寒冷干燥。"